# Funding Campaigns

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from os.path import join
import wget, os
from glob import glob
import urllib


# pandas by default only shows a few columns, we want them all!
%load_ext autoreload

%autoreload 2

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [31]:
path = '../data'

In [7]:
web_data = requests.get('https://webrobots.io/kickstarter-dataset/')

In [ ]:
path_4_source = join(path, 'kickstarter', 'raw_files')
path_4_source

In [34]:
# with open(join(path_4_source, 'indiegogo.html'), 'w') as html:
#     html.write(web_data.text)

In [ ]:
# html_doc = join(path_4_source, 'indiegogo.html')
# html_doc

In [8]:
# with open(html_doc, 'r') as file: 
soup = BeautifulSoup(web_data.text, 'html.parser')

In [16]:
list_of_links = [link.get('href') for link in soup.find_all('a')]
our_list = pd.Series(list_of_links, name='links').dropna().reset_index(drop=True)
our_list.head(2)

0                 #content
1    https://webrobots.io/
Name: links, dtype: object

In [17]:
csv_files = our_list.str.endswith('Z.zip')
data_we_need = our_list[csv_files]
data_we_need.head()

17    https://s3.amazonaws.com/weruns/forfun/Kicksta...
19    https://s3.amazonaws.com/weruns/forfun/Kicksta...
21    https://s3.amazonaws.com/weruns/forfun/Kicksta...
23    https://s3.amazonaws.com/weruns/forfun/Kicksta...
25    https://s3.amazonaws.com/weruns/forfun/Kicksta...
Name: links, dtype: object

In [44]:
def get_me_all_data(data_we_need, path_to_files):
    """
    NOTE: Only use this function if you intend to download ALL!! of the data.
    
    Arguments:
    urls: pandas series with the links to iterate over
    path_to_files: path where you would like to save your files at
    countries_unique: iterable with the countries where Airbnb operates
    """
    
    new_dir = check_or_add(path_to_files)
    
    for data in data_we_need: # iterate over the links we want
        
        name = data.split('/')[-1]
        
#             date = data[1]['dates'].replace('-', '_')

        try: # we first try to download the file with wget
            wget.download(data, join(new_dir, name))
        except:
            try: # if wget doesn't work, we try with urllib
                urllib.request.urlretrieve(data, join(new_dir, name))
            except:
                continue # if urllib doesn't work, we move on to the next one

In [45]:
%%time

get_me_all_data(data_we_need, path_4_source)

CPU times: user 5.44 s, sys: 4 s, total: 9.44 s
Wall time: 4min 37s


In [61]:
ddf = dd.read_csv(join('..', 'data', 'indiegogo', 'raw_files', '*Z.zip'), blocksize=None)
len(ddf.columns)

21